In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import rrcf
from rrcf.shingle import shingle

data = np.load("taxi_all.npz")

num_trees = 200
forest = []
sample_size = 1000
data_dim = 48
data_index = []

# Anamolies from the paper
time_stamps = [np.datetime64(i) for i in data["dates"]]

independence_day = (time_stamps.index(np.datetime64('2014-07-04 00:00:00')),
                    time_stamps.index(np.datetime64('2014-07-07 00:00:00')))
labor_day        = (time_stamps.index(np.datetime64('2014-09-01 00:00:00')),
                    time_stamps.index(np.datetime64('2014-09-02 00:00:00')))
labor_day_parade = (time_stamps.index(np.datetime64('2014-09-06 00:00:00')),
                    time_stamps.index(np.datetime64('2014-09-07 00:00:00')))
NYC_marathon     = (time_stamps.index(np.datetime64('2014-11-02 00:00:00')),
                    time_stamps.index(np.datetime64('2014-11-03 00:00:00')))
thanksgiving     = (time_stamps.index(np.datetime64('2014-11-27 00:00:00')),
                    time_stamps.index(np.datetime64('2014-11-28 00:00:00')))
christmas        = (time_stamps.index(np.datetime64('2014-12-25 00:00:00')),
                    time_stamps.index(np.datetime64('2014-12-26 00:00:00')))
new_year         = (time_stamps.index(np.datetime64('2015-01-01 00:00:00')),
                    time_stamps.index(np.datetime64('2015-01-02 00:00:00')))
blizzard         = (time_stamps.index(np.datetime64('2015-01-26 00:00:00')),
                    time_stamps.index(np.datetime64('2015-01-28 00:00:00')))

anamolies = [independence_day, labor_day, labor_day_parade, NYC_marathon, thanksgiving, christmas, new_year, blizzard]

time_stamps = np.asarray(time_stamps)

# make the inital trees
data_stream = shingle(data["passengers"], data_dim)
inital_data = next(data_stream)
inital_data = np.vstack((inital_data, next(data_stream)))
inital_data = np.vstack((inital_data, next(data_stream)))
# add 3 data points
data_index = [0, 1, 2]

forest = [rrcf.RCTree(inital_data) for i in range(0, num_trees)]
codisp = np.zeros((len(data["passengers"]), num_trees))
for tree in forest:
    codisp[1, forest.index(tree)] = tree.codisp(1)
    codisp[2, forest.index(tree)] = tree.codisp(2)

for i in range(48, len(data["passengers"])-3):
    if len(data_index) >= sample_size:
        # forget the point
        for tree in forest:
            tree.forget_point(data_index[0])
        data_index.pop(0)

    data_index.append(i+3)
    # insert the new point
    d = next(data_stream)
    for tree in forest:
        tree.insert_point(np.asarray(d), data_index[-1])
        # compute the codisp
        codisp[i, forest.index(tree)] = tree.codisp(data_index[-1])

np.save("codisp_trees_"+str(sample_size), codisp)

plt.subplot(2,1,1)
plt.plot(codisp.mean(axis=1))
plt.subplot(2,1,2)
plt.plot(data["dates"], data["passengers"])
plt.show()